# Scheduling

The scheduler is responsible for managing the entirety of the process life cycle and for selecting the next process to run. In the next few sections we will describe the process life cycle and several methods for ordering runnable processes. Before we discuss details we need to define a few new terms. Earlier we described ho we will approach virtualizing the CPU, by allowing a process to run for an amount of time and then switching for another process and then repeating these steps as long as there are processes that need to run. We allot a bounded amount of time for a process to run in a given turn and this bounded time is called a time slice.

## The Process Life Cycle

Processes are created in one of two ways: the operating system creates the first process (called init in UNIX-like systems) during boot up, or any process can use a system call (e.g., `fork()`, `clone()`, `NtCreateProcess()`, etc). Processes can also be terminated in three ways: by itself by calling `exit()`, `ExitProcess()`, or equivalent, due to an error (e.g., segmentation fault, division by zero, etc.), or by another process calling `kill()`, `TerminateProcess()`, or similar.

Once created a process has the following life cycle.
    
```{figure} ../images/scheduling/ProcessLifecycle.png
---
name: lifecycle
---
The process life cycle showing the possible transitions between the states.
```

As shown in {numref}`lifecycle`, there are 3 possible states for a process and all the valid stat transitions are represented by the arrows. A running process is one that is currently active on a CPU. A running process can move to either the 'Ready' or 'Blocked' state. To move to the Blocked state (labeled 1), a process must take an action that will require 'significant[^sig-def]' time to complete. Some examples of blocking operations are: issuing a read from disk or from a network socket that does not have data ready, trying to acquire certain kinds of locks that are already owned by another process or thread, etc. A Running process that takes an action that will block immediately moves to the Blocked state, allowing another process to use the CPU while it waits To move to Ready from Running (labeled 2), the running process has to either yield the remainder of its allotted time slice, or be stopped by the OS after exhausting its slice. Moving from Ready to Running (labeled 3) requires that the OS to select the process as the next to execute and place it on a processor. The final transition we have not discussed is from Blocked to Ready (labeled 4), this transition happens when the blocking request is now complete or ready. 

[^sig-def]: The word significant is intentionally vague here because operations that were considered 'blocking' 10 years ago may not be any longer, and ones which are blocking today may not be in a decade.